# 使用@property

为了限制score的范围，可以通过一个set_score()方法来设置成绩，再通过一个get_score()来获取成绩，这样，在set_score()方法里，就可以检查参数：

In [10]:
class Student(object):
    
    def get_score(self):
        return self._score
    
    def set_score(self,value):
        if not isinstance(value,int):
            raise ValueError('Score must be an interger!')
        if value < 0 or value > 100:
            raise ValueError('Score must be between 0~100')
            
        self._score = value
        

s = Student()
s.set_score(90)
s.get_score()
#s.set_score(999)

90

如果要让内部属性不被外部访问，可以把属性的名称前加上两个下划线\__，在Python中，实例的变量名如果以__开头，就变成了一个私有变量（private），只有内部可以访问，外部不能访问

有些时候，你会看到以一个下划线开头的实例变量名，比如_name，这样的实例变量外部是可以访问的，但是，按照约定俗成的规定，当你看到这样的变量时，意思就是，“虽然我可以被访问，但是，请把我视为私有变量，不要随意访问”。

但是，上面的调用方法又略显复杂，没有直接用属性这么直接简单。有没有既能检查参数，又可以用类似属性这样简单的方式来访问类的变量呢？还记得装饰器（decorator）可以给函数动态加上功能吗？对于类的方法，装饰器一样起作用。Python内置的@property装饰器就是负责把一个方法变成属性调用的：

In [18]:
class Student(object):
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self,value):
        if not isinstance(value,int):
            raise ValueError('Score must be an interger!')
        if value < 0 or value > 100:
            raise ValueError('Score must be between 0~100')
            
        self._score = value
        

s = Student()
s.score = 60 # 实际事转化s.set_score(60)
s.score # 实际事转化s.get_score()

# s.score = 999

60

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作。 

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性：

In [21]:
class Student(object):
    
    @property
    def birth(self):
        return self._birth
    
    @birth.setter
    def birth(self,value):
        self._birth = value
    
    @property
    def age(self):
        return 2018-self._birth
    
s = Student()
s.birth = 28
s.age

1990

上面的birth是可读写属性，而age就是一个只读属性，因为age可以根据birth和当前时间计算出来。

# 两句话掌握python最难知识点——元类
https://segmentfault.com/a/1190000011447445

学懂元类，你只需要知道两句话：

### 道生一，一生二，二生三，三生万物

### 我是谁？我从哪来里？我要到哪里去？


在python世界，拥有一个永恒的道，那就是“type”，请记在脑海中，type就是道。如此广袤无垠的python生态圈，都是由type产生出来的。

道生一，一生二，二生三，三生万物。

道 即是 type

一 即是 metaclass(元类，或者叫类生成器)

二 即是 class(类，或者叫实例生成器)

三 即是 instance(实例)

万物 即是 实例的各种属性与方法，我们平常使用python时，调用的就是它们。


In [1]:
# 创建一个Hello类，拥有属性 say_heelo --- 二的起源
class Hello():
    def say_hello(self,name='world'):
        print('Hello,{}'.format(name))

# 从 Hello 类创建一个实例 hello  --- 二生三
hello = Hello()

# 使用 hello 调用方法 say_hello --- 三生万物
hello.say_hello() 

Hello,world


这就是一个标准的“二生三，三生万物”过程。 从类到我们可以调用的方法，用了这两步。

那我们不由自主要问，类从何而来呢？回到代码的第一行。
class Hello其实是一个函数的“语义化简称”，只为了让代码更浅显易懂，它的另一个写法是：

这样的写法，就和之前的Class Hello写法作用完全相同,建实例并调用,输出效果也一样

In [2]:
# 假如我们有一个函数 fn
def fn(self,name='world'):
    print('Hello,{}'.format(name))

# 通过 type 创建 Hello 类   --- 道生二
Hello = type('Hello',(object,),dict(say_hello=fn))

hello = Hello()

hello.say_hello()

Hello,world


`Hello = type('Hello', (object,), dict(say_hello=fn)) `

type(类名, 父类的元组（针对继承的情况，可以为空），包含属性的字典（名称和值）)

注意它的三个参数！暗合人类的三大永恒命题：**我是谁，我从哪里来，我要到哪里去.**

第一个参数：我是谁。类名

第二个参数：我从哪里来。也就是我的“父类”。

第三个参数：我要到哪里去。需要调用的方法和属性包含到一个字典里，再作为参数传入。


```
class Hello(object){
# class 后声明“我是谁”
# 小括号内声明“我来自哪里”
# 中括号内声明“我要到哪里去”
    def say_hello(){
        
    }
}
```

造物主，可以直接创造单个的人，但这是一件苦役。造物主会先创造“人”这一物种，再批量创造具体的个人。并将三大永恒命题，一直传递下去。

**“道”可以直接生出“二”，但它会先生出“一”，再批量地制造“二”。**

**type可以直接生成类（class），但也可以先生成元类（metaclass），再使用元类批量定制类（class）。**

## 元类——道生一，一生二

一般来说，元类均被命名后缀为Metalass。想象一下，我们需要一个可以自动打招呼的元类，它里面的类方法呢，有时需要say_Hello，有时需要say_Hi，有时又需要say_Sayolala，有时需要say_Nihao。

如果每个内置的say_xxx都需要在类里面声明一次，那将是多么可怕的苦役！ 不如使用元类来解决问题。

以下是创建一个专门“打招呼”用的元类代码：

In [ ]:
class SayMetaClass(type):
    
    def __new__(cls,name,bases,attrs):
        attrs['say_'+name]=lambda self,value,saying=name:print('{},{}!'.foramt(saying,value))
        return type.__new__(cls,name,bases,attrs)
    
    

记住两点：

1、元类是由“type”衍生而出，所以父类需要传入type。**【道生一，所以一必须包含道】**

2、元类的操作都在 \__new__中完成，它的第一个参数是将创建的类，之后的参数即是三大永恒命题：我是谁，我从哪里来，我将到哪里去。 它返回的对象也是三大永恒命题，接下来，这三个参数将一直陪伴我们。

在\__new__中，我只进行了一个操作，就是这个

`attrs['say_'+name] = lambda self,value,saying=name:print('{},{}!'.foramt(saying,value))`

其中匿名函数等同于
```
def niming(self,value,saying=name):
    print('{},{}!'.foramt(saying,value)

```

\__new__()方法接收到的参数依次是：

当前准备创建的类的对象；

类的名字；

类继承的父类集合；

类的方法集合。